# Calculate ILAMB Data
Creates annual, regridded data as well as month of max and climatology, from raw ILAMB data. Uses a config file (e.g. ILAMB_obs.cfg) to get information about where ILAMB data is located and what the files, models, and variable names are.

In [1]:
import os
import warnings
import logging
import xarray as xr

import fates_calibration_library.utils as utils
import fates_calibration_library.ilamb_functions as ilamb
import fates_calibration_library.analysis_functions as analysis

logging.basicConfig(level=logging.WARNING)
logger = logging.getLogger(__name__)

In [2]:
# config file for ILAMB data
ilamb_cfg = '/glade/work/afoster/FATES_calibration/fates_calibration_library/configs/ILAMB_obs.yaml'

# land area file
land_area_ds = os.path.join("/glade/derecho/scratch/afoster/archive",
                            "ctsm60SP_bigleaf_fullgrid/lnd/hist",
                            "ctsm60SP_bigleaf_fullgrid.clm2.h0.0001-02-01-00000.nc")

In [3]:
# config dictionary for running functions
config_dict = {
    'top_dir': '/glade/campaign/cesm/community/lmwg/diag/ILAMB/DATA',  # location of raw ILAMB data
    'regrid_dir': '/glade/work/afoster/FATES_calibration/observations/ILAMB_obs/regridded',  # where to put regridded ILAMB data
    'regrid_tag': '2deg',
    'out_dir': '/glade/work/afoster/FATES_calibration/observations/ILAMB_obs',  # where to put compiled ILAMB data
    'user': 'afoster@ucar.edu',  # user
    'clobber': False,  # overwrite files?
}

In [ ]:
import importlib
importlib.reload(ilamb)

In [4]:
# read in the ILAMB data dictionary
ilamb_dict = utils.get_config_file(ilamb_cfg)

In [5]:
# create a target grid to regrid to
target_grid = analysis.create_target_grid(land_area_ds, 'FSR')

In [ ]:
# loop through dictionary and regrid the ILAMB observations
# script will skip files that already exist unless clobber: True
ilamb.regrid_all_ilamb_data(config_dict, ilamb_dict, target_grid)

In [6]:
# loop through dictionary and process the ILAMB observations
# script will skip files that already exist unless clobber: True
ilamb.get_all_ilamb_data(config_dict, ilamb_dict)

/glade/work/afoster/conda-envs/fates_calibration/lib/python3.11/site-packages/xarray/conventions.py:436: SerializationWarning: variable 'lai' has multiple fill values {1e+36, -999.0}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(
/glade/work/afoster/conda-envs/fates_calibration/lib/python3.11/site-packages/xarray/conventions.py:436: SerializationWarning: variable 'nee' has multiple fill values {-9999.0, -999.0}, decoding all values to NaN.
  new_vars[k] = decode_cf_variable(


In [7]:
# compile all datasets into one dataset
compiled_ds = ilamb.compile_ilamb_datasets(config_dict['out_dir'], ilamb_dict,
                                           target_grid.area)
compiled_ds.to_netcdf(os.path.join('/glade/work/afoster/FATES_calibration',
                                    'observations/all_ILAMB_obs.nc'), mode="w")